In [ ]:
import os
import torch
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
from time import perf_counter

from sklearn.metrics import precision_recall_fscore_support, accuracy_score, jaccard_score
import torchnet as tnt

from sklearn.model_selection import KFold, StratifiedKFold
from tqdm import notebook as tqdm

import image_preprocessing
import inference_utils

# GLOBAL SETTINGS
matplotlib.rcParams['figure.figsize'] = [5, 5]  
np.set_printoptions(precision=2, suppress=True)  # Array print precision

In [ ]:
# PATHS TO TRAINING DATA
trainingdata_path = r'C:\Users\dd\Documents\NATUR_CUNI\additional_projects\prace\ETRAINEE\data\LH_202008_54bands_9cm.tif'
referencedata_path = r'C:\Users\dd\Documents\NATUR_CUNI\additional_projects\prace\ETRAINEE\data\LH_202008_reference.tif'

In [ ]:
loaded_raster = image_preprocessing.read_gdal(trainingdata_path, referencedata_path)
print(f'Tiled imagery shape {loaded_raster["imagery"].shape}')
print(f'Tiled reference shape {loaded_raster["reference"].shape}')

In [ ]:
tile_shape = (128, 128)
overlap = 64
offset = (0, 0)

dataset_tiles = image_preprocessing.tile_training(loaded_raster, tile_shape, overlap, offset)
print(f'Tiled imagery shape {dataset_tiles["imagery"].shape}')
print(f'Tiled reference shape {dataset_tiles["reference"].shape}')

In [ ]:
filtered_tiles = image_preprocessing.filter_useful_tiles(dataset_tiles, nodata_vals=[65535], is_training=True)
print(f'Filtered imagery shape {filtered_tiles["imagery"].shape}')
print(f'Filtered reference shape {filtered_tiles["reference"].shape}')

In [ ]:
preprocessed_tiles, unique, counts = image_preprocessing.normalize_tiles_3d(filtered_tiles, nodata_vals=[65535], is_training=True)
print(f'Preprocessed imagery shape {preprocessed_tiles["imagery"].shape}')
print(f'Preprocessed reference shape {preprocessed_tiles["reference"].shape}')

dataset = tnt.dataset.TensorDataset([preprocessed_tiles['imagery'], preprocessed_tiles['reference']])
print(dataset)

print(f'Class labels: \n{unique}\n')
print(f'Number of pixels in a class: \n{counts}')

In [ ]:
class SpectroSpatialNet(nn.Module):
    """3D Spectral-Spatial CNN for semantic segmentation."""

    def __init__(self, args):
        """
        Initialize the SpectroSpatial model.

        n_channels, int, number of input channel
        size_e, int list, size of the feature maps of convs for the encoder
        size_d, int list, size of the feature maps of convs for the decoder
        n_class = int,  the number of classes
        """
        # necessary for all classes extending the module class
        super(SpectroSpatialNet, self).__init__()

        self.maxpool = nn.MaxPool3d(2, 2, return_indices=False)
        self.dropout = nn.Dropout3d(p=0.5, inplace=True)

        self.n_channels = args['n_channel']
        self.size_e = args['size_e']
        self.size_d = args['size_d']
        self.n_class = args['n_class']

        # Encoder layer definitions
        def c_en_3d(in_ch, out_ch, k_size=3, pad=1, pad_mode='zeros',
                    bias=False):
            """Create default conv layer for the encoder."""
            return nn.Sequential(nn.Conv3d(in_ch, out_ch, kernel_size=k_size,
                                           padding=pad, padding_mode=pad_mode,
                                           bias=bias),
                                 nn.BatchNorm3d(out_ch), nn.ReLU())

        def c_de_2d(in_ch, out_ch, k_size=3, pad=1, pad_mode='zeros',
                    bias=False):
            """Create default conv layer for the decoder."""
            return nn.Sequential(nn.Conv2d(in_ch, out_ch, kernel_size=k_size,
                                           padding=pad, padding_mode=pad_mode,
                                           bias=bias),
                                 nn.BatchNorm2d(out_ch), nn.ReLU())

        self.c1 = c_en_3d(self.n_channels, self.size_e[0])
        self.c2 = c_en_3d(self.size_e[0], self.size_e[1])
        self.c3 = c_en_3d(self.size_e[1], self.size_e[2])
        self.c4 = c_en_3d(self.size_e[2], self.size_e[3])
        self.c5 = c_en_3d(self.size_e[3], self.size_e[4])
        self.c6 = c_en_3d(self.size_e[4], self.size_e[5])

        self.trans1 = nn.ConvTranspose2d(self.size_d[0], self.size_d[1],
                                      kernel_size=2, stride=2)
        self.c7 = c_de_2d(self.size_d[1], self.size_d[2])
        self.c8 = c_de_2d(self.size_d[2], self.size_d[3])
        self.trans2 = nn.ConvTranspose2d(self.size_d[3], self.size_d[4],
                                      kernel_size=2, stride=2)
        self.c9 = c_de_2d(self.size_d[4], self.size_d[5])
        self.c10 = c_de_2d(self.size_d[5], self.size_d[6])

        # Final classifying layer
        self.classifier = nn.Conv2d(self.size_d[6], self.n_class,
                                    1, padding=0)

        # Weight initialization
        self.c1[0].apply(self.init_weights)
        self.c2[0].apply(self.init_weights)
        self.c3[0].apply(self.init_weights)
        self.c4[0].apply(self.init_weights)
        self.c5[0].apply(self.init_weights)
        self.c6[0].apply(self.init_weights)

        self.c7[0].apply(self.init_weights)
        self.c8[0].apply(self.init_weights)

        self.c9[0].apply(self.init_weights)
        self.c10[0].apply(self.init_weights)
        self.classifier.apply(self.init_weights)

        # Put the model on GPU memory
        if torch.cuda.is_available():
            self.cuda()
            torch.backends.cudnn.enabled = True
            torch.backends.cudnn.benchmark = True

    def init_weights(self, layer):  # gaussian init for the conv layers
        """Initialise layer weights."""
        nn.init.kaiming_normal_(
            layer.weight, mode='fan_out', nonlinearity='relu')

    def forward(self, input_data):
        """Define model structure."""
        # Encoder
        # level 1
        x1 = self.c2(self.c1(input_data))
        x2 = self.maxpool(x1)
        # level 2
        x3 = self.c4(self.c3(x2))
        x4 = self.maxpool(x3)
        # level 3
        x5 = self.c6(self.c5(x4))
        # Decoder
        # Level 3
        y5 = torch.flatten(x5, start_dim=1, end_dim=2)
        # level 2
        y4 = self.trans1(y5)
        y3 = self.c8(self.c7(y4))
        # level 1
        y2 = self.trans2(y3)
        y1 = self.c10(self.c9(y2))
        # Output
        out = self.classifier(self.dropout(y1))
        return out

In [ ]:
def augment(obs, g_t):
    """the data augmentation function, introduces random noise and rotation"""
    sigma, clip= 0.01, 0.03 

    # Random noise
    rand = torch.clamp(torch.mul(sigma, torch.randn([1, 1, 54, tile_shape[0],tile_shape[1]])), -clip, clip)
    obs = torch.add(obs, rand)

    # Random rotation 0 90 180 270 degree
    n_turn = np.random.randint(4) #number of 90 degree turns, random int between 0 and 3
    obs = torch.rot90(obs, n_turn, dims=(3,4))
    g_t = torch.rot90(g_t, n_turn, dims=(1,2))

    return obs, g_t

In [ ]:
def train(model, optimizer, args):
    """train for one epoch"""
    model.train() #switch the model in training mode
  
    #the loader function will take care of the batching
    loader = torch.utils.data.DataLoader(dataset, batch_size=args['batch_size'], sampler=args['train_subsampler'])
    loader = tqdm.tqdm(loader, ncols=500)
  
    #will keep track of the loss
    loss_meter = tnt.meter.AverageValueMeter()

    for index, (tiles, gt) in enumerate(loader):
    
        optimizer.zero_grad() #put gradient to zero

        tiles, gt = augment(tiles, gt)

        pred = model(tiles.cuda()) #compute the prediction

        loss = nn.functional.cross_entropy(pred.cpu(),gt, weight=args['class_weights'])
        loss.backward() #compute gradients

        for p in model.parameters(): #we clip the gradient at norm 1
            p.grad.data.clamp_(-1, 1) #this helps learning faster

        optimizer.step() #one SGD step
        loss_meter.add(loss.item())
        
    return loss_meter.value()[0]

def eval(model, sampler):
    """eval on test/validation set"""
  
    model.eval() #switch in eval mode
  
    loader = torch.utils.data.DataLoader(dataset, batch_size=1, sampler=sampler)
    loader = tqdm.tqdm(loader, ncols=500)
  
    loss_meter = tnt.meter.AverageValueMeter()

    with torch.no_grad():
        for index, (tiles, gt) in enumerate(loader):
            pred = model(tiles.cuda())
            loss = nn.functional.cross_entropy(pred.cpu(), gt)
            loss_meter.add(loss.item())

    return loss_meter.value()[0]


def train_full(args):
    """The full training loop"""

    #initialize the model
    model = SpectroSpatialNet(args)

    print(f'Total number of parameters: {sum([p.numel() for p in model.parameters()])}')
  
    #define the Adam optimizer
    optimizer = optim.Adam(model.parameters(), lr=args['lr'])
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=args['scheduler_milestones'],
                                               gamma=args['scheduler_gamma'])
  
    train_loss = np.empty(args['n_epoch'])
    test_epochs = []
    test_loss = []

    for i_epoch in range(args['n_epoch']):
        #train one epoch
        print(f'Epoch #{str(i_epoch+1)}')
        train_loss[i_epoch] = train(model, optimizer, args)
        scheduler.step()

        # Periodic testing on the validation set
        if (i_epoch == args['n_epoch'] - 1) or ((i_epoch + 1) % args['n_epoch_test'] == 0):
            print('Evaluation')
            loss_test = eval(model, args['test_subsampler'])
            test_epochs.append(i_epoch + 1)
            test_loss.append(loss_test)
            
    plt.figure(figsize=(10, 10))
    plt.subplot(1,1,1,ylim=(0,5), xlabel='Epoch #', ylabel='Loss')
    plt.plot([i+1 for i in range(args['n_epoch'])], train_loss, label='Training loss')
    plt.plot(test_epochs, test_loss, label='Validation loss')
    plt.legend()
    plt.show()
    print(train_loss)
    print(test_loss)
    args['loss_test'] = test_loss[-1]
    
    return model

## Training a 3D network

In [ ]:
args = { #Dict to store all model parameters
    'n_channel': 1,
    'n_class': len(unique),
    'size_e': [16,16,32,32,32,32],
    'size_d': [416,64,64,64,64,32,32],
    
    'crossval_nfolds': 3,
    'n_epoch_test': 2,          #periodicity of evaluation on test set
    'scheduler_milestones': [60,80,90],
    'scheduler_gamma': 0.3,
    'class_weights': torch.tensor([0.0, 0.202, 0.341, 0.033, 0.16, 0.14, 0.03, 0.013, 0.023, 0.058]),

    'n_epoch': 100,
    'lr': 5e-6,
    'batch_size': 1,
}
model_save_folder = r'C:\Users\dd\Documents\NATUR_CUNI\additional_projects\prace\ETRAINEE\data\models\3D'

print(f'''Number of models to be trained:
    {args['crossval_nfolds']}
Number of spectral channels:
    {args['n_channel']}
Initial learning rate:
    {args['lr']}
Batch size:
    {args['batch_size']}
Number of training epochs:
    {args['n_epoch']}''')

In [ ]:
## Training a 3D network
kfold = KFold(n_splits = args['crossval_nfolds'], shuffle=True)
trained_models = []
for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):
    print(f'Training starts for model number {str(fold+1)}')
    
    a = perf_counter()
    args['train_subsampler'] = torch.utils.data.SubsetRandomSampler(train_ids)
    args['test_subsampler'] = torch.utils.data.SubsetRandomSampler(test_ids)
    
    trained_models.append((train_full(args), args['loss_test']))
    
    state_dict_path = os.path.join(model_save_folder, f'fold_{str(fold)}.pt')
    torch.save(trained_models[fold][0].state_dict(), state_dict_path)
    print(f'Model saved to: {state_dict_path}')
    print(f'Training finished in {str(perf_counter()-a)}s')
    print('\n\n')

print(f'Resulting loss for individual folds: \n{[i for _, i in trained_models]}')
print(f'Mean loss across all folds: \n{np.mean([i for _, i in trained_models])}')

## Saving and reusing a trained model

In [ ]:
# Path to the state_dictionary
state_dict_path = r'C:\Users\dd\Documents\NATUR_CUNI\additional_projects\prace\ETRAINEE\data\models\fold_2.pt'

Save a model to state_dict_path:

In [ ]:
# Save a trained model state_dictionary
torch.save(trained_model.state_dict(), state_dict_path)

Reuse a model at state_dict_path:

In [ ]:
# Parameters for model definition

args = {} #stores the parameters
args['n_class'] = 10
args['n_channel'] = 1
args['size_e'] =  np.divide([64,64,128,128,256,256,512,512,1024,1024],        32).astype(np.int32)
args['size_d'] = np.divide(np.multiply([1024,512,512,512,256,256,256,128,128,128,64,64], 3), 32).astype(np.int32)
args['cuda'] = True


# Load a trained model state_dictionary
#state_dict_path = 'd:/studenti/JD/ETRAINEE/models/fold_0.pt'
model = SpectroSpatialNet(args)
#state_dict_path = 'D:/Studenti/JD/Jeseniky_2020/models/KrakonosNet_jes_1e-3_200epochs.pt'
#model = KrakonosNet(args['n_channel'], args['size_e'], args['size_d'], args['n_class'], args['cuda'])
model.load_state_dict(torch.load(state_dict_path))
model.eval()

## Export results
Results are not georeferenced – use ArcPy_georeference_results.py for georeferencing and combining into a single raster

In [ ]:
source_path = r'C:\Users\dd\Documents\NATUR_CUNI\additional_projects\prace\ETRAINEE\data\LH_202008_54pasem_9cm.tif'
tile_shape = (256, 256)
overlap = 128
offset_topleft = (0, 0)

start = perf_counter()
raster_orig = image_preprocessing.read_gdal_with_geoinfo(source_path, offset_topleft)
print(raster_orig['geoinfo'])

dataset_full_tiles = image_preprocessing.run_tiling_dims(raster_orig['imagery'], out_shape=tile_shape, 
                                                    out_overlap=overlap, offset=offset_topleft)
dataset_full = image_preprocessing.normalize_tiles_3d(dataset_full_tiles, nodata_vals=[0])
dataset = tnt.dataset.TensorDataset(dataset_full['imagery'])
end = perf_counter()
print(f'Loading the imagery took {end - start} seconds.')

print(dataset_full_tiles['imagery'].shape)
print(dataset_full_tiles['dimensions'])

In [ ]:
output_path = r'C:\Users\dd\Documents\NATUR_CUNI\additional_projects\prace\ETRAINEE\data\test_result_3d.tif'

start = perf_counter()
arr_class = inference_utils.combine_tiles_2d(model, dataset, tile_shape, overlap, dataset_full_tiles['dimensions'])
print(np.unique(arr_class, return_counts=True))
inference_utils.export_result(output_path, arr_class, raster_orig['geoinfo'])
end = perf_counter()
print(f'The processing took {end - start} seconds.')
plt.imshow(arr_class)